In [3]:
"""Module for fetching data from the SEC EDGAR Archives"""
import json
import os
import re
import requests
from typing import List, Optional, Tuple, Union
import sys

if sys.version_info < (3, 8):
    from typing_extensions import Final
else:
    from typing import Final

import webbrowser

from ratelimit import limits, sleep_and_retry

VALID_FILING_TYPES: Final[List[str]] = [
    "10-K",
    "10-Q",
    "S-1",
    "10-K/A",
    "10-Q/A",
    "S-1/A",
]

SEC_ARCHIVE_URL: Final[str] = "https://www.sec.gov/Archives/edgar/data"
SEC_SEARCH_URL: Final[str] = "http://www.sec.gov/cgi-bin/browse-edgar"
SEC_SUBMISSIONS_URL = "https://data.sec.gov/submissions"


def get_filing(
    cik: Union[str, int], accession_number: Union[str, int], company: str, email: str
) -> str:
    """Fetches the specified filing from the SEC EDGAR Archives. Conforms to the rate
    limits specified on the SEC website.
    ref: https://www.sec.gov/os/accessing-edgar-data"""
    session = _get_session(company, email)
    return _get_filing(session, cik, accession_number)


@sleep_and_retry
@limits(calls=10, period=1)
def _get_filing(
    session: requests.Session, cik: Union[str, int], accession_number: Union[str, int]
) -> str:
    """Wrapped so filings can be retrieved with an existing session."""
    url = archive_url(cik, accession_number)
    response = session.get(url)
    response.raise_for_status()
    return response.text


@sleep_and_retry
@limits(calls=10, period=1)
def get_cik_by_ticker(session: requests.Session, ticker: str) -> str:
    """Gets a CIK number from a stock ticker by running a search on the SEC website."""
    cik_re = re.compile(r".*CIK=(\d{10}).*")
    url = _search_url(ticker)
    response = session.get(url, stream=True)
    response.raise_for_status()
    results = cik_re.findall(response.text)
    return str(results[0])


@sleep_and_retry
@limits(calls=10, period=1)
def get_forms_by_cik(session: requests.Session, cik: Union[str, int]) -> dict:
    """Gets retrieves dict of recent SEC form filings for a given cik number."""
    json_name = f"CIK{cik}.json"
    response = session.get(f"{SEC_SUBMISSIONS_URL}/{json_name}")
    response.raise_for_status()
    content = json.loads(response.content)
    recent_forms = content["filings"]["recent"]
    form_types = {k: v for k, v in zip(recent_forms["accessionNumber"], recent_forms["form"])}
    return form_types


def _get_recent_acc_num_by_cik(
    session: requests.Session, cik: Union[str, int], form_types: List[str]
) -> Tuple[str, str]:
    """Returns accession number and form type for the most recent filing for one of the
    given form_types (AKA filing types) for a given cik."""
    retrieved_form_types = get_forms_by_cik(session, cik)
    for acc_num, form_type_ in retrieved_form_types.items():
        if form_type_ in form_types:
            return _drop_dashes(acc_num), form_type_
    raise ValueError(f"No filings found for {cik}, looking for any of: {form_types}")


def get_recent_acc_by_cik(
    cik: str,
    form_type: str,
    company: Optional[str] = None,
    email: Optional[str] = None,
) -> Tuple[str, str]:
    """Returns (accession_number, retrieved_form_type) for the given cik and form_type.
    The retrieved_form_type may be an amended version of requested form_type, e.g. 10-Q/A for 10-Q.
    """
    session = _get_session(company, email)
    return _get_recent_acc_num_by_cik(session, cik, _form_types(form_type))


def get_recent_cik_and_acc_by_ticker(
    ticker: str,
    form_type: str,
    company: Optional[str] = None,
    email: Optional[str] = None,
) -> Tuple[str, str, str]:
    """Returns (cik, accession_number, retrieved_form_type) for the given ticker and form_type.
    The retrieved_form_type may be an amended version of requested form_type, e.g. 10-Q/A for 10-Q.
    """
    session = _get_session(company, email)
    cik = get_cik_by_ticker(session, ticker)
    acc_num, retrieved_form_type = _get_recent_acc_num_by_cik(session, cik, _form_types(form_type))
    return cik, acc_num, retrieved_form_type


def get_form_by_ticker(
    ticker: str,
    form_type: str,
    allow_amended_filing: Optional[bool] = True,
    company: Optional[str] = None,
    email: Optional[str] = None,
) -> str:
    """For a given ticker, gets the most recent form of a given form_type."""
    session = _get_session(company, email)
    cik = get_cik_by_ticker(session, ticker)
    return get_form_by_cik(
        cik, form_type, allow_amended_filing=allow_amended_filing, company=company, email=email
    )


def _form_types(form_type: str, allow_amended_filing: Optional[bool] = True):
    """Potentialy expand to include amended filing, e.g.:
    "10-Q" -> "10-Q/A"
    """
    assert form_type in VALID_FILING_TYPES
    if allow_amended_filing and not form_type.endswith("/A"):
        return [form_type, f"{form_type}/A"]
    else:
        return [form_type]


def get_form_by_cik(
    cik: str,
    form_type: str,
    allow_amended_filing: Optional[bool] = True,
    company: Optional[str] = None,
    email: Optional[str] = None,
) -> str:
    """For a given CIK, returns the most recent form of a given form_type. By default
    an amended version of the form_type may be retrieved (allow_amended_filing=True).
    E.g., if form_type is "10-Q", the retrived form could be a 10-Q or 10-Q/A.
    """
    session = _get_session(company, email)
    acc_num, _ = _get_recent_acc_num_by_cik(
        session, cik, _form_types(form_type, allow_amended_filing)
    )
    text = _get_filing(session, cik, acc_num)
    return text


def open_form(cik, acc_num):
    """For a given cik and accession number, opens the index page in default browser for the
    associated SEC form"""
    acc_num = _drop_dashes(acc_num)
    webbrowser.open_new_tab(f"{SEC_ARCHIVE_URL}/{cik}/{acc_num}/{_add_dashes(acc_num)}-index.html")


def open_form_by_ticker(
    ticker: str,
    form_type: str,
    allow_amended_filing: Optional[bool] = True,
    company: Optional[str] = None,
    email: Optional[str] = None,
):
    """For a given ticker, opens the index page in default browser for the most recent form of a
    given form_type."""
    session = _get_session(company, email)
    cik = get_cik_by_ticker(session, ticker)
    acc_num, _ = _get_recent_acc_num_by_cik(
        session, cik, _form_types(form_type, allow_amended_filing)
    )
    open_form(cik, acc_num)


def archive_url(cik: Union[str, int], accession_number: Union[str, int]) -> str:
    """Builds the archive URL for the SEC accession number. Looks for the .txt file for the
    filing, while follows a {accession_number}.txt format."""
    filename = f"{_add_dashes(accession_number)}.txt"
    accession_number = _drop_dashes(accession_number)
    return f"{SEC_ARCHIVE_URL}/{cik}/{accession_number}/{filename}"


def _search_url(cik: Union[str, int]) -> str:
    search_string = f"CIK={cik}&Find=Search&owner=exclude&action=getcompany"
    url = f"{SEC_SEARCH_URL}?{search_string}"
    return url


def _add_dashes(accession_number: Union[str, int]) -> str:
    """Adds the dashes back into the accession number"""
    accession_number = str(accession_number)
    return f"{accession_number[:10]}-{accession_number[10:12]}-{accession_number[12:]}"


def _drop_dashes(accession_number: Union[str, int]) -> str:
    """Converts the accession number to the no dash representation."""
    accession_number = str(accession_number).replace("-", "")
    return accession_number.zfill(18)


def _get_session(company: Optional[str] = None, email: Optional[str] = None) -> requests.Session:
    """Creates a requests sessions with the appropriate headers set. If these headers are not
    set, SEC will reject your request.
    ref: https://www.sec.gov/os/accessing-edgar-data"""
    if company is None:
        company = os.environ.get("SEC_API_ORGANIZATION")
    if email is None:
        email = os.environ.get("SEC_API_EMAIL")
    assert company
    assert email
    session = requests.Session()
    session.headers.update(
        {
            "User-Agent": f"{company} {email}",
            "Content-Type": "text/html",
        }
    )
    return session


In [4]:
text = get_form_by_ticker(
    'aapl',
    '10-K',
    company='Unstructured Technologies',
    email='support@unstructured.io'
)

## Cleaning up the .XML file

In [5]:
from unstructured.documents.html import HTMLDocument

html_document = HTMLDocument.from_string(text).doc_after_cleaners(skip_headers_and_footers=True, skip_table_text=True)


In [6]:
for element in html_document.pages[0].elements[::]: #only from the first page
    print(element)
    print("\n")

UNITED STATES


SECURITIES AND EXCHANGE COMMISSION


Washington, D.C. 20549


FORM 10-K


(Mark One)


ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934


For the fiscal year ended September 24, 2022


or


TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934


For the transition period from


to


Commission File Number:


001-36743


Apple Inc.


(Exact name of Registrant as specified in its charter)


(408) 996-1010


(Registrant’s telephone number, including area code)


Securities registered pursuant to Section 12(b) of the Act:


Securities registered pursuant to Section 12(g) of the Act:  None


Indicate by check mark if the Registrant is a well-known seasoned issuer, as defined in Rule 405 of the Securities Act.


Yes


No


Indicate by check mark if the Registrant is not required to file reports pursuant to Section 13 or Section 15(d) of the Act.


Yes


No


Indicate by check mark whether the Registrant (1

In [7]:
html_document.pages[0].elements[::]

### Check to see if a section could be a title

In [8]:
from unstructured.nlp.partition import is_possible_title

#Getting the name of a title from the HTMLTitle object
text_title = html_document.pages[0].elements[22].text

print(text_title)

#Check if that title could be a title or not
is_possible_title(text_title)


Indicate by check mark if the Registrant is not required to file reports pursuant to Section 13 or Section 15(d) of the Act.


False

### Making bricks

In [9]:
import re
from unstructured.documents.elements import Title

In [10]:
ITEM_TITLE_RE = re.compile(
    r"(?i)item \d{1,3}(?:[a-z]|\([a-z]\))?(?:\.)?(?::)?"
)

In [11]:
def is_10k_item_title(title: str) -> bool:
    """Determines if a title corresponds to a 10-K item heading."""
    return ITEM_TITLE_RE.match(title) is not None

In [12]:
for element in html_document.elements:
    if isinstance(element, Title) and is_10k_item_title(element.text):
        print(element)

Item 1.    Business
Item 1A.    Risk Factors
Item 1B.    Unresolved Staff Comments
Item 2.    Properties
Item 3.    Legal Proceedings
Item 4.    Mine Safety Disclosures
Item 7.    Management’s Discussion and Analysis of Financial Condition and Results of Operations
Item 7A.    Quantitative and Qualitative Disclosures About Market Risk
Item 8.    Financial Statements and Supplementary Data
Item 9.    Changes in and Disagreements with Accountants on Accounting and Financial Disclosure
Item 9A.    Controls and Procedures
Item 9B.    Other Information
Item 9C.    Disclosure Regarding Foreign Jurisdictions that Prevent Inspections
Item 10.    Directors, Executive Officers and Corporate Governance
Item 11.    Executive Compensation
Item 13.    Certain Relationships and Related Transactions, and Director Independence
Item 14.    Principal Accountant Fees and Services
Item 15.    Exhibit and Financial Statement Schedules
Item 16.    Form 10-K Summary


### Cleaning bricks

In [13]:
from unstructured.cleaners.core import clean_extra_whitespace

In [14]:
titles = []
for element in html_document.elements:
    #element.text = clean_extra_whitespace(element.text)
    if isinstance(element, Title) and is_10k_item_title(element.text):
        titles.append(element)
        print(element)

print(titles)

Item 1.    Business
Item 1A.    Risk Factors
Item 1B.    Unresolved Staff Comments
Item 2.    Properties
Item 3.    Legal Proceedings
Item 4.    Mine Safety Disclosures
Item 7.    Management’s Discussion and Analysis of Financial Condition and Results of Operations
Item 7A.    Quantitative and Qualitative Disclosures About Market Risk
Item 8.    Financial Statements and Supplementary Data
Item 9.    Changes in and Disagreements with Accountants on Accounting and Financial Disclosure
Item 9A.    Controls and Procedures
Item 9B.    Other Information
Item 9C.    Disclosure Regarding Foreign Jurisdictions that Prevent Inspections
Item 10.    Directors, Executive Officers and Corporate Governance
Item 11.    Executive Compensation
Item 13.    Certain Relationships and Related Transactions, and Director Independence
Item 14.    Principal Accountant Fees and Services
Item 15.    Exhibit and Financial Statement Schedules
Item 16.    Form 10-K Summary
[<unstructured.documents.html.HTMLTitle obj

In [15]:
section_info = []
for i, el in enumerate(html_document.elements):
  #print(el)
  #print(titles[0])
  if el.id == titles[0].id:
    break
first_title_index = i
for i in range(first_title_index, first_title_index+30):
  section_info.append(html_document.elements[i].text)


In [17]:
{type(el) for el in html_document.elements}

{unstructured.documents.html.HTMLListItem,
 unstructured.documents.html.HTMLNarrativeText,
 unstructured.documents.html.HTMLText,
 unstructured.documents.html.HTMLTitle}

### Using OpenAI

In [18]:
import os
import openai

OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]


In [19]:

human_template = f"""You are analyzing a 10-K document for me. Enclosed by the word START and END, I will give you the data of the 10-K and you will answer a question based ONLY on information you find enclosed in the triple quotes, so you will use no outside information.

Here is the 10-K data:
START
{section_info}
END

This is the question: How does teh company generate its revenue? What are its main products or services?
"""

human_message_prompt = {"role": "user", "content": human_template}


out = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-16k",
    temperature=0,
    messages=[
        human_message_prompt,
    ],
    api_key=OPENAI_API_KEY
)
json_string = out["choices"][0]["message"]["content"]

print(json_string)
#data = json.loads(json_string)

The company generates its revenue by designing, manufacturing, and marketing smartphones, personal computers, tablets, wearables, and accessories, as well as selling a variety of related services. Its main products include the iPhone line of smartphones, the Mac line of personal computers, the iPad line of multipurpose tablets, and the Wearables, Home, and Accessories category, which includes AirPods and Apple TV.
